In [8]:
from icecream import ic
import pandas as pd
from dateutil.relativedelta import relativedelta
# from dhs_preprocessing_functions import *
from pandarallel import pandarallel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
# Initialization
pandarallel.initialize()

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [9]:
# folder: path = "/mnt/datadisk/data/surveys/DHS_final_raw_data/"
# work_dir = "/mnt/datadisk/data/Projects/water/pickles/"
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
# min_version = 3
overwrite_pqt = True
dataset_type = 'HR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 25 surveys
group_by_col = 'adm2_gaul'

urban_rural_all_mode = 'R' # 'all', 'U', 'R'

in_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}_{urban_rural_all_mode}.pkl"
print(in_f)
# categorical_in_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}{add_str}.pkl"
fewsNet_f = f"{input_dir}fewsnet_gaul_adm2.csv"

out_f = f"{input_dir}5_grouped_df_V3_{dataset_type}_{group_by_col}_joined_with_ipc_{urban_rural_all_mode}.pkl"

/mnt/datadisk/data/Projects/water/inputs/grouped_df_V3_HR_adm2_gaul_R.pkl


In [10]:
df = pd.read_pickle(in_f)
df

Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; adm2_gaul                                              
AOGE52FL        Ambaca                        Kuanza Norte          Angola   
                Amboim                          Cuanza Sul          Angola   
                Andulo                                 Bie          Angola   
                Baia Farta                        Benguela          Angola   
                Balombo                           Benguela          Angola   
...                                                    ...             ...   
ZWGE72FL        Umzingwane              Matabeleland South        Zimbabwe   
                Uzumba-Maramba-Pfungwe    Mashonaland East        Zimbabwe   
                Zaka                              Masvingo        Zimbabwe   
                Zvimba                    Mashonaland West        Zimbabwe   
                Zvishavane                        Midlands        Zimbabwe   

                                        Meta; year  Meta; month Meta; date  \
Meta; GEID_init Meta; adm2_gaul                                              
AOGE52FL        Ambaca                        2007            3 2007-03-06   
                Amboim                        2006           12 2006-12-11   
                Andulo                        2007            1 2007-01-08   
                Baia Farta                    2006           11 2006-11-27   
                Balombo                       2006           12 2006-12-07   
...                                            ...          ...        ...   
ZWGE72FL        Umzingwane                    2015            7 2015-07-10   
                Uzumba-Maramba-Pfungwe        2015           10 2015-10-12   
                Zaka                          2015           10 2015-10-19   
                Zvimba                        2015           11 2015-11-25   
                Zvishavane                    2015           10 2015-10-28   

                                        Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; adm2_gaul                                                    
AOGE52FL        Ambaca                                                       1.0   
                Amboim                                                       1.0   
                Andulo                                                       1.0   
                Baia Farta                                                   1.0   
                Balombo                                                      1.0   
...                                                                          ...   
ZWGE72FL        Umzingwane                                                   0.0   
                Uzumba-Maramba-Pfungwe                                       0.0   
                Zaka                                                         0.0   
                Zvimba                                                       0.0   
                Zvishavane                                                   0.0   

                                        Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; adm2_gaul                                                   
AOGE52FL        Ambaca                                                      0.0   
                Amboim                                                      0.0   
                Andulo                                                      0.0   
                Baia Farta                                                  0.0   
                Balombo                                                     0.0   
...                                                                         ...   
ZWGE72FL        Umzingwane                                                  0.0   
                Uzumba-Maramba-Pfungwe                                      0.0   
                Zaka                                                        0.0   
       

In [11]:
food_security_df = pd.read_csv(fewsNet_f)
food_security_df.head()
ic(food_security_df.columns.to_list())
food_security_df = food_security_df[['adm2_name', 'adm1_name', 'adm0_name', 'month', 'year', 'CS', 'HA0']]
food_security_df['HA0'] = food_security_df['HA0'].fillna(0)
food_security_df['IPC + food help'] = food_security_df['CS'] + food_security_df['HA0']
food_security_df.drop(columns=['HA0'], inplace=True)
food_security_df.columns = ['adm2_name', 'adm1_name', 'adm0_name', 'month', 'year', 'IPC', 'IPC + food help']
print(len(food_security_df))
food_security_df.sample(10)

/tmp/ipykernel_3884421/3621970319.py:1: DtypeWarning: Columns (26,27,28,29,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  food_security_df = pd.read_csv(fewsNet_f)
ic| food_security_df.columns.to_list(): ['OBJECTID',
                                         'adm2_code',
                                         'adm2_name',
                                         'str2_year',
                                         'exp2_year',
                                         'adm1_code',
                                         'adm1_name',
                                         'status',
                                         'disp_area',
                                         'adm0_code',
                                         'adm0_name',
                                         'shape_leng',
                                         'FID_1',
                                         'Shape_Le_1',
                                   

104386


,adm2_name,adm1_name,adm0_name,month,year,IPC,IPC + food help
33103,Awka South,Anambra,Nigeria,10,2015,1.0,1.0
12226,Soba,Kaduna,Nigeria,10,2011,1.0,1.0
65194,Talodi,Southern Kordofan,Sudan,4,2012,4.0,4.0
101874,Nyanga,Manicaland,Zimbabwe,10,2019,3.0,3.0
81523,Salh,Taizz,Yemen,2,2019,3.0,4.0
35418,Kabalaye,Tandjile Est,Chad,6,2016,1.0,1.0
26773,Kayes,Kayes,Mali,7,2014,1.0,1.0
47479,Bogoro,Bauchi,Nigeria,6,2019,1.0,1.0
92077,Monze,Southern,Zambia,7,2011,1.0,1.0
102232,Panda,Inhambane,Mozambique,2,2020,2.0,3.0


In [12]:
def aggregate_ipc(row, food_security_df, start_delta, end_delta, y_m_indicator):
    # Create datetime objects for the start and end dates
    in_date = row['Meta; date']
    if y_m_indicator == 'm':
        start_date = in_date + relativedelta(months=start_delta)
        end_date = in_date + relativedelta(months=end_delta)
    elif y_m_indicator == 'y':
        start_date = in_date + relativedelta(years=start_delta)
        end_date = in_date + relativedelta(years=end_delta)
    else:
        raise ValueError(f"y_m_indicator should be 'm' or 'y', not {y_m_indicator}")
    # Filter the data for the given time span and adm2_gaul
    # print(row.name)
    if group_by_col == 'adm2_gaul':
        adm2 = row.name[1]
    else:
        adm2 = row['Meta; adm2_gaul']
    
    df_time_span = food_security_df[(food_security_df['date'] >= start_date) & (food_security_df['date'] < end_date) & 
                                    (food_security_df['adm2_name'] == adm2) &
                                    (food_security_df['adm1_name'] == row['Meta; adm1_gaul']) &
                                    (food_security_df['adm0_name'] == row['Meta; adm0_gaul'])]


    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: mean'] = df_time_span['IPC'].mean()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: max'] = df_time_span['IPC'].max()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: min'] = df_time_span['IPC'].min()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: median'] = round(df_time_span['IPC'].median(), 0)

    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: mean'] = df_time_span['IPC + food help'].mean()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: max'] = df_time_span['IPC + food help'].max()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: min'] = df_time_span['IPC + food help'].min()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: median'] = round(df_time_span['IPC + food help'].median(), 0)

    return row

# Convert year and month to datetime
food_security_df['date'] = pd.to_datetime(food_security_df[['year', 'month']].assign(day=1))
print(food_security_df.shape)
print(df.shape)
print('Meta; date' in df.columns)

for start_delta, end_delta, m_y_ind in [(0, 5, 'm'), (6, 12, 'm'), (1, 2, 'y'), (2, 4, 'y'), (4, 6, 'y'), (6, 7, 'y'), (6, 10, 'y'), (10, 15, 'y'), (15, 20, 'y'), (20, 25, 'y'),
                                        (0, 1, 'y'), (0, 2, 'y'), (2, 6, 'y'), (6, 12, 'y'), (12, 20, 'y')]:
    df = df.parallel_apply(aggregate_ipc, axis=1, food_security_df=food_security_df, start_delta=start_delta, 
                                            end_delta=end_delta, y_m_indicator=m_y_ind)

df[df['FS; IPC + FH: 15-20y: mean'].notna()]

(104386, 8)
(12963, 1437)
True


Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; adm2_gaul                                              
BFGE32FL        Bam                            Centre-nord    Burkina Faso   
                Banwa                    Boucle Du Mouhoun    Burkina Faso   
                Bazega                          Centre-sud    Burkina Faso   
                Boulgou                         Centre-est    Burkina Faso   
                Boulkiemde                    Centre-ouest    Burkina Faso   
...                                                    ...             ...   
ZWGE52FL        Umzingwane              Matabeleland South        Zimbabwe   
                Uzumba-Maramba-Pfungwe    Mashonaland East        Zimbabwe   
                Zaka                              Masvingo        Zimbabwe   
                Zvimba                    Mashonaland West        Zimbabwe   
                Zvishavane                        Midlands        Zimbabwe   

                                        Meta; year  Meta; month  \
Meta; GEID_init Meta; adm2_gaul                                   
BFGE32FL        Bam                           1999            2   
                Banwa                         1999            1   
                Bazega                        1999            2   
                Boulgou                       1998           12   
                Boulkiemde                    1999            2   
...                                            ...          ...   
ZWGE52FL        Umzingwane                    2005            9   
                Uzumba-Maramba-Pfungwe        2005           10   
                Zaka                          2005           11   
                Zvimba                        2005           12   
                Zvishavane                    2005           12   

                                                Meta; date  \
Meta; GEID_init Meta; adm2_gaul                              
BFGE32FL        Bam                    1999-02-04 00:00:00   
                Banwa                  1999-01-21 12:00:00   
                Bazega                 1999-02-07 00:00:00   
                Boulgou                1998-12-21 00:00:00   
                Boulkiemde             1999-02-03 00:00:00   
...                                                    ...   
ZWGE52FL        Umzingwane             2005-09-23 00:00:00   
                Uzumba-Maramba-Pfungwe 2005-10-19 00:00:00   
                Zaka                   2005-11-24 00:00:00   
                Zvimba                 2005-12-19 00:00:00   
                Zvishavane             2005-12-07 12:00:00   

                                        Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; adm2_gaul                                                    
BFGE32FL        Bam                                                          0.0   
                Banwa                                                        0.0   
                Bazega                                                       0.0   
                Boulgou                                                      0.0   
                Boulkiemde                                                   0.0   
...                                                                          ...   
ZWGE52FL        Umzingwane                                                   0.0   
                Uzumba-Maramba-Pfungwe                                       0.0   
                Zaka                                                         0.0   
                Zvimba                                                       0.0   
                Zvishavane                                                   0.0   

                                        Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; adm2_gaul                                                   
BFGE32FL        Bam                                                         0.0   
       

In [13]:
# save the dataframe
df.to_pickle(out_f)
df.to_csv(out_f.replace('.pkl', '.csv'), index=False)

In [14]:
df

Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; adm2_gaul                                              
AOGE52FL        Ambaca                        Kuanza Norte          Angola   
                Amboim                          Cuanza Sul          Angola   
                Andulo                                 Bie          Angola   
                Baia Farta                        Benguela          Angola   
                Balombo                           Benguela          Angola   
...                                                    ...             ...   
ZWGE72FL        Umzingwane              Matabeleland South        Zimbabwe   
                Uzumba-Maramba-Pfungwe    Mashonaland East        Zimbabwe   
                Zaka                              Masvingo        Zimbabwe   
                Zvimba                    Mashonaland West        Zimbabwe   
                Zvishavane                        Midlands        Zimbabwe   

                                        Meta; year  Meta; month Meta; date  \
Meta; GEID_init Meta; adm2_gaul                                              
AOGE52FL        Ambaca                        2007            3 2007-03-06   
                Amboim                        2006           12 2006-12-11   
                Andulo                        2007            1 2007-01-08   
                Baia Farta                    2006           11 2006-11-27   
                Balombo                       2006           12 2006-12-07   
...                                            ...          ...        ...   
ZWGE72FL        Umzingwane                    2015            7 2015-07-10   
                Uzumba-Maramba-Pfungwe        2015           10 2015-10-12   
                Zaka                          2015           10 2015-10-19   
                Zvimba                        2015           11 2015-11-25   
                Zvishavane                    2015           10 2015-10-28   

                                        Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; adm2_gaul                                                    
AOGE52FL        Ambaca                                                       1.0   
                Amboim                                                       1.0   
                Andulo                                                       1.0   
                Baia Farta                                                   1.0   
                Balombo                                                      1.0   
...                                                                          ...   
ZWGE72FL        Umzingwane                                                   0.0   
                Uzumba-Maramba-Pfungwe                                       0.0   
                Zaka                                                         0.0   
                Zvimba                                                       0.0   
                Zvishavane                                                   0.0   

                                        Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; adm2_gaul                                                   
AOGE52FL        Ambaca                                                      0.0   
                Amboim                                                      0.0   
                Andulo                                                      0.0   
                Baia Farta                                                  0.0   
                Balombo                                                     0.0   
...                                                                         ...   
ZWGE72FL        Umzingwane                                                  0.0   
                Uzumba-Maramba-Pfungwe                                      0.0   
                Zaka                                                        0.0   
       